In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import Dataset, DataLoader, random_split

# Dataset definition
class CustomDataset(Dataset):
    def __init__(self, data_dir):
        self.data_dir = data_dir
        self.files = [f for f in os.listdir(data_dir) if f.endswith('.txt')]
    
    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, idx):
        filename = self.files[idx]
        filepath = os.path.join(self.data_dir, filename)
        data = np.loadtxt(filepath)
        inputs = data[:4]
        target = int(data[4])  # Convert target to integer for classification
        return torch.tensor(inputs, dtype=torch.float32), torch.tensor(target, dtype=torch.long)  # Use torch.long for classification targets

# MLP model definition
class MLP(nn.Module):
    def __init__(self, num_classes):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(4, 128)
        self.fc2 = nn.Linear(128, 256)
        self.fc3 = nn.Linear(256, 64)
        self.fc4 = nn.Linear(64, 32)
        self.fc5 = nn.Linear(32, num_classes)  # Use num_classes as the output size
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return x

# Function to save the model
def save_model(model, path):
    torch.save(model.state_dict(), path)

# Function to load the model
def load_model(path, num_classes, device):
    model = MLP(num_classes).to(device)
    model.load_state_dict(torch.load(path))
    model.eval()  # Set the model to evaluation mode
    return model

# Main function
if __name__ == "__main__":

    data_dir = "data_classification/all"
    dataset = CustomDataset(data_dir)
    device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
    
    # Split dataset into training and validation sets (4:1 ratio)
    train_size = int(0.8 * len(dataset))
    val_size = len(dataset) - train_size
    train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
    
    batch_size = 64  # You can experiment with 128, 256, etc.
    
    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)
    
    # Model, loss function, optimizer
    num_classes = 2  # Number of classes in your classification problem
    model = MLP(num_classes).to(device)
    criterion = nn.CrossEntropyLoss()  # Use CrossEntropyLoss for classification
    optimizer = optim.Adam(model.parameters(), lr=0.0001)
    
    # TensorBoard writer
    writer = SummaryWriter(log_dir='run_class/mpl_class_5_batch256_lr0.0001-2')
    
    # Training loop
    num_epochs = 500  # Adjust this as necessary
    best_val_loss = float('inf')
    best_model_path = "best_model.pth"
    
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for i, (inputs, targets) in enumerate(train_loader):
            inputs, targets = inputs.to(device), targets.to(device)
            
            # Zero the parameter gradients
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            
            # Backward pass and optimize
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
        
        # Calculate average training loss for this epoch
        avg_train_loss = running_loss / len(train_loader)
        
        # Log training loss to TensorBoard
        writer.add_scalar('Training Loss', avg_train_loss, epoch)
        
        # Validation loop
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, targets in val_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total += targets.size(0)
                correct += (predicted == targets).sum().item()
        
        # Calculate average validation loss for this epoch
        avg_val_loss = val_loss / len(val_loader)
        
        # Calculate validation accuracy
        val_accuracy = correct / total
        
        # Log validation loss and accuracy to TensorBoard
        writer.add_scalar('Validation Loss', avg_val_loss, epoch)
        writer.add_scalar('Validation Accuracy', val_accuracy, epoch)
        
        # Print average loss for this epoch
        if epoch % 10 == 9:
            print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}")
        
        # Save the best model
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            save_model(model, best_model_path)
    
    # Close the writer
    writer.close()
    
    # Load the best model for prediction
    model = load_model(best_model_path, num_classes, device)
    
    # Example prediction
    test_input = torch.tensor([0.5, 0.3, 0.2, 0.1], dtype=torch.float32).to(device)
    model.eval()
    with torch.no_grad():
        prediction = model(test_input.unsqueeze(0))  # Add batch dimension
        predicted_class = torch.argmax(prediction, dim=1)
    print(f"Prediction: {predicted_class.item()}")


Epoch [10/500], Train Loss: 0.4379369644880295, Val Loss: 0.8806531377874625
Epoch [20/500], Train Loss: 0.43627999064922335, Val Loss: 0.8772934378145602
Epoch [30/500], Train Loss: 0.4338857795000076, Val Loss: 0.873531307465733
Epoch [40/500], Train Loss: 0.42408533990383146, Val Loss: 0.8519981541572669
Epoch [50/500], Train Loss: 0.38669112474918366, Val Loss: 0.7723422819813981
Epoch [60/500], Train Loss: 0.3696509324073792, Val Loss: 0.7407200949641463
Epoch [70/500], Train Loss: 0.36232737529277803, Val Loss: 0.7279524622252955
Epoch [80/500], Train Loss: 0.35820523414611816, Val Loss: 0.7183061236390671
Epoch [90/500], Train Loss: 0.35569348828792574, Val Loss: 0.7110483813971377
Epoch [100/500], Train Loss: 0.35310817174911496, Val Loss: 0.7053517265060839
Epoch [110/500], Train Loss: 0.35090491518974304, Val Loss: 0.7057188954977943
Epoch [120/500], Train Loss: 0.3494901102781296, Val Loss: 0.702143887932689
Epoch [130/500], Train Loss: 0.34764352645874025, Val Loss: 0.69612